# Titanic Dataset - Assignment 1

This is a small exploratory data analysis for the Titanic dataset and a strategy for inputing missing variables into the "Age" column of the data by Daniel Marranzini.

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("data/Titanic/train.csv", index_col="PassengerId")
df.head(10)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S


In [5]:
# Percentage of survivors out of total number of passengers.
survivors_total = df["Survived"].mean()
print("{:.2%}".format(survivors_total), "of passengers survived.")

38.38% of passengers survived.


In [6]:
# Percentage of men and women out of total number of passengers.
sex_total = df.groupby("Sex")["Name"].count()/len(df)
print("{:.2%}".format(sex_total["male"]), "of passengers were men.")
print("{:.2%}".format(sex_total["female"]), "of passengers were women.")

64.76% of passengers were men.
35.24% of passengers were women.


In [7]:
# Percentage of survivors by sex.
sex_survived = df.groupby("Sex")["Survived"].mean()
print("{:.2%}".format(sex_survived["male"]), "of men survived.")
print("{:.2%}".format(sex_survived["female"]), "of women survived.")

18.89% of men survived.
74.20% of women survived.


In [8]:
# Sex distribution of survivors.
dist_survived = df.loc[df["Survived"]==1].groupby("Sex")["Survived"].count()/len(df.loc[df["Survived"]==1])
print("{:.2%}".format(dist_survived["male"]), "of survivors were men.")
print("{:.2%}".format(dist_survived["female"]), "of survivors were women.")

31.87% of survivors were men.
68.13% of survivors were women.


In [9]:
# The probability of being male/female and surviving/not surviving.
p_survival = pd.crosstab(df["Sex"], df["Survived"])/len(df)
p_survival

Survived,0,1
Sex,,
female,0.090909,0.261504
male,0.525253,0.122334


In [10]:
# Checking for correctness, rounded to avoid python decimal point errors.
round(p_survival.values.sum(), 2)

1.0

In [11]:
# The probability of survival/not survival of all combinations of sex and class.
p_combined = pd.crosstab([df["Sex"], df["Pclass"]], df["Survived"])/len(df)
p_combined

Survived              0         1
Sex    Pclass                    
female 1       0.003367  0.102132
       2       0.006734  0.078563
       3       0.080808  0.080808
male   1       0.086420  0.050505
       2       0.102132  0.019080
       3       0.336700  0.052750

In [12]:
# Checking for correctness again.
round(p_combined.values.sum(), 2)

1.0

# Inputing Missing Values

For predicting the survival the *exact* age isn't the key, but whether they are elderly or children can greatly affect the outcome. For this we could use the title clearly available in each name along with their class to better determine the passenger's age. 

In [13]:
# Extracting the title from the name.
df["Title"] = df.Name.str.split(".").str.get(0).str.split(", ").str.get(1)
df["Title"].unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Don', 'Rev', 'Dr', 'Mme', 'Ms',
       'Major', 'Lady', 'Sir', 'Mlle', 'Col', 'Capt', 'the Countess',
       'Jonkheer'], dtype=object)

In [19]:
# Checking to see if our grouping is logical.
df.groupby(["Title", "Pclass"])["Age"].median()

Title         Pclass
Capt          1         70.0
Col           1         58.0
Don           1         40.0
Dr            1         46.5
              2         38.5
Jonkheer      1         38.0
Lady          1         48.0
Major         1         48.5
Master        1          4.0
              2          1.0
              3          4.0
Miss          1         30.0
              2         24.0
              3         18.0
Mlle          1         24.0
Mme           1         24.0
Mr            1         40.0
              2         31.0
              3         26.0
Mrs           1         41.5
              2         32.0
              3         31.0
Ms            2         28.0
Rev           2         46.5
Sir           1         49.0
the Countess  1         33.0
Name: Age, dtype: float64

Here we group by the new title column we created and the passenger's class to extract the median age of passengers within those groups. We can also see above that these groupings make sense, passengers with titles like Master are clearly children, while adults with titles like Mrs. are usually older than adults with titles like Miss. The "Pclass" also provides a clear distinction with the median age rising as the eliteness of the class rises as well. So we will go ahead with the transformation and filling our missing values.

In [46]:
df_filling = df.groupby(["Title", "Pclass"])["Age"].transform("median")
df_filling

PassengerId
1      26.0
2      41.5
3      18.0
4      41.5
5      26.0
       ... 
887    46.5
888    30.0
889    18.0
890    40.0
891    26.0
Name: Age, Length: 891, dtype: float64

In [49]:
df["Age"] = df["Age"].fillna(df_filling)
df

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
PassengerId,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr
...,...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,Rev
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,Miss
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,18.0,1,2,W./C. 6607,23.4500,NaN,S,Miss


My code would fail here and I'd need an extra fallback if one of the passengers with unique titles such as "Jonkheer" or "the Countess" was missing their age, but that is not the case for this dataset.

In [27]:
# Checking if we missed any null values.
df["Age"].isnull().any()

False

In [28]:
# Dropping our title column and presenting our finished dataframe.
df = df.drop("Title", axis=1)
df.head(10)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
6,0,3,"Moran, Mr. James",male,26.0,0,0,330877,8.4583,NaN,Q
7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
